# Урок 4: KV-Cache Quantization

При обработке длинных контекстов память GPU забивается не весами модели, а кэшем ключей (Keys) и значений (Values). Сжатие кэша позволяет увеличить контекст с 8k до 128k без покупки новых GPU.

## 1. Математическая теория

### 1.1. Расчет объема
$$Size_{KV} = 2 \cdot L \cdot H \cdot d \cdot S \cdot P$$
Где: $L$ - слои, $H$ - головы, $d$ - размер головы, $S$ - длина контекста, $P$ - точность (байт). 
Для Llama-8B это ~2 ГБ на каждые 8к токенов.

### 1.2. Методы из обзора:
*   **KVQuant (Hooper et al., 2024):** Применяет «неравномерное» квантование. Метод использует калибровку, чтобы найти оптимальные уровни квантования для каждого канала Keys, учитывая их специфическое распределение.
*   **KIVI (Liu et al., 2024):** Асимметричное решение. Метод квантует Keys по каналам (per-channel), а Values — по токенам (per-token). Это позволяет сжать кэш до 2 бит при сохранении высокого качества генерации.
*   **WKVQuant (Yue et al., 2024):** Использует кросс-блочную регуляризацию, учитывая, как ошибка квантования кэша в одном слое влияет на последующие слои внимания.

---

In [ ]:
import torch
from src.model import GPTLanguageModel

def estimate_kv_cache_size_bytes(seq_len, n_layer=6, n_head=6, n_embd=384, precision=4):
    # 2 (K и V) * layers * head_dim * seq_len * precision_bytes
    head_dim = n_embd // n_head
    return 2 * n_layer * n_head * head_dim * seq_len * (precision / 8)

print(f"nanoGPT KV-Cache (1024 tokens, 4-bit): {estimate_kv_cache_size_bytes(1024)/1024:.1f} KB")

## 2. Промышленная реализация: Transformers Quantized Cache
В новых версиях `transformers` появилась поддержка 4-битного квантования кэша «из коробки».

In [ ]:
from transformers import AutoModelForCausalLM, QuantizedCacheConfig
try:
    cache_config = QuantizedCacheConfig(nbits=4, axis=0)
    model = AutoModelForCausalLM.from_pretrained("tiny-llama", torch_dtype=torch.float16)
    # Во время генерации просто указываем cache_implementation
    # model.generate(..., cache_implementation="quantized", cache_config=cache_config)
    print("Llama Track: Поддержка 4-битного кэша (QuantizedCache) сконфигурирована.")
except Exception as e:
    print(f"Ошибка: {e}")